In [1]:
import nlg

In [ ]:
pip install spacy==2.0.13

In [2]:
import spacy
spacy.__version__

'3.7.4'

In [ ]:
!pip install inflect

In [ ]:
######## Restart Kernal if needed ##########

In [ ]:
!python -m spacy download en_core_web_sm

In [3]:
from nlg.utils import load_spacy_model

In [4]:
from nlg import grammar

In [5]:
from nlg.search import templatize

In [6]:
import pandas as pd

In [7]:
df=pd.read_csv("actors.csv")

In [21]:
df

,Category,name,rating,votes
0,Actors,Humphery,0.570197,109
1,Actors,Cary,0.998602,142
2,Actors,Grant,0.600000,120
3,Actors,James,0.988370,108
4,Actresses,Bette,0.500000,86
5,Actresses,Ingrid,0.400000,63


In [9]:
from gramex.data import filter as gfilter
sort_args={'_sort':['-rating']}
xdf=gfilter(df,sort_args.copy())

In [10]:
xdf

,Category,name,rating,votes
1,Actors,Cary,0.998602,142
3,Actors,James,0.988370,108
2,Actors,Grant,0.600000,120
0,Actors,Humphery,0.570197,109
4,Actresses,Bette,0.500000,86
5,Actresses,Ingrid,0.400000,63


In [11]:
nlp=load_spacy_model()

In [ ]:
text=nlp("James is the actor with the highest rating.")
nugget=templatize(text,sort_args,df)

In [13]:
nugget

{% set fh_args = {"_sort": ["-rating"]}  %}
{% set df = U.gfilter(orgdf, fh_args.copy()) %}
{% set fh_args = U.sanitize_fh_args(fh_args, orgdf) %}
{# Do not edit above this line. #}
{{ df["name"].iloc[1] }} is the {{ G.singular(df["Category"].iloc[3]).lower() }} with the highest {{ df.columns[2] }}.

In [14]:
print(nugget.render(df))

b'    James is the actor with the highest rating.'


In [15]:
df["name"].iloc[1]

'Cary'

In [16]:
df["Category"].iloc[3]

'Actors'

In [17]:
df.columns[2] 

'rating'

In [20]:
df

,Category,name,rating,votes
0,Actors,Humphery,0.570197,109
1,Actors,Cary,0.998602,142
2,Actors,Grant,0.600000,120
3,Actors,James,0.988370,108
4,Actresses,Bette,0.500000,86
5,Actresses,Ingrid,0.400000,63


In [22]:
import numpy as np
np.random.seed(12345)

fake_ratings = df.copy()
fake_ratings['rating'] = np.random.rand(df.shape[0])

In [23]:
fake_ratings.sort_values('rating', ascending=False).head()

,Category,name,rating,votes
0,Actors,Humphery,0.929616,109
5,Actresses,Ingrid,0.595545,63
4,Actresses,Bette,0.567725,86
1,Actors,Cary,0.316376,142
3,Actors,James,0.204560,108


In [24]:
nugget.render(fake_ratings)

b'    Ingrid is the actor with the highest rating.'

In [25]:
print(nugget.template)

{% set fh_args = {"_sort": ["-rating"]}  %}
{% set df = U.gfilter(orgdf, fh_args.copy()) %}
{% set fh_args = U.sanitize_fh_args(fh_args, orgdf) %}
{# Do not edit above this line. #}
{{ df["name"].iloc[1] }} is the {{ G.singular(df["Category"].iloc[3]).lower() }} with the highest {{ df.columns[2] }}.


In [26]:
nugget.variables

{rating: {{ df.columns[2] }},
 James: {{ df["name"].iloc[1] }},
 actor: {{ G.singular(df["Category"].iloc[3]).lower() }}}

In [35]:
var = nugget.get_var('actor')
var.set_expr('df["Category"].iloc[0]')
var

{{ G.singular(df["Category"].iloc[0]).lower() }}

In [36]:
nugget.variables

{rating: {{ df.columns[2] }},
 James: {{ df["name"].iloc[1] }},
 actor: {{ G.singular(df["Category"].iloc[0]).lower() }}}

In [38]:
nugget.render(fake_ratings)

b'    Ingrid is the actor with the highest rating.'

In [10]:
######################################## Using Rule Based Method #############################################

In [13]:
import pandas as pd
from datetime import datetime

# Create DataFrame from the provided table
data = {
    'Country': ['HK', 'HK', 'HK', 'HK', 'HK', 'HK'],
    'Channel': ['Internet', 'Internet', 'Internet', 'Mobile', 'Mobile', 'Mobile'],
    'Month_Year': ['Dec-23', 'Jan-24', 'Feb-24', 'Dec-23', 'Jan-24', 'Feb-24'],
    'Successful': [673480, 630152, 673480, 123456, 673480, 876767],
    'UnSuccessful': [673480, 630152, 673480, 123456, 673480, 876767],
    'Total': [673480, 630152, 673480, 123456, 673480, 876767],
    'Success Percentage': [99.9, 95.8, 99.87, 99.9, 99.87, 99.87]
}

df = pd.DataFrame(data)

# Convert Month_Year column to datetime format
df['Month_Year'] = pd.to_datetime(df['Month_Year'], format='%b-%y')

# Define NLG function to generate summary
def generate_summary(data):
    summary = []

    # Check if latest month success percentage is greater than 99.67%
    latest_month = data['Month_Year'].max()
    latest_data = data[data['Month_Year'] == latest_month]
    for _, row in latest_data.iterrows():
        if row['Success Percentage'] > 99.67:
            summary.append(f"Last month, {latest_month.strftime('%b-%y')}, target is achieved for {row['Channel']} Channel.")

    # Analyze success percentage trends for each channel over the past three months
    for channel, channel_data in data.groupby('Channel'):
        trend_data = channel_data.tail(3)
        min_success_percentage = trend_data['Success Percentage'].min()
        for _, row in trend_data.iterrows():
            if row['Success Percentage'] == min_success_percentage:
                summary.append(f"From the past three months for {channel} Channel, {row['Month_Year'].strftime('%b-%y')} shows the lowest Success Percentage.")

        # Analyze total volume trends for each channel over the past three months
        total_volumes = trend_data['Total'].tolist()
        max_volume_month = trend_data.loc[trend_data['Total'].idxmax(), 'Month_Year']
        min_volume_month = trend_data.loc[trend_data['Total'].idxmin(), 'Month_Year']
        if max_volume_month != min_volume_month:
            summary.append(f"From the past three months for {channel} Channel, {max_volume_month.strftime('%b-%y')} has the highest total volume, and {min_volume_month.strftime('%b-%y')} has the lowest total volume.")

    return summary

# Generate summary
summary = generate_summary(df)

# Print summary
for s in summary:
    print(s)


Last month, Feb-24, target is achieved for Internet Channel.
Last month, Feb-24, target is achieved for Mobile Channel.
From the past three months for Internet Channel, Jan-24 shows the lowest Success Percentage.
From the past three months for Internet Channel, Feb-24 has the highest total volume, and Jan-24 has the lowest total volume.
From the past three months for Mobile Channel, Jan-24 shows the lowest Success Percentage.
From the past three months for Mobile Channel, Feb-24 shows the lowest Success Percentage.
From the past three months for Mobile Channel, Feb-24 has the highest total volume, and Jan-24 has the lowest total volume.


In [ ]:
################## Filering based on Country ############

In [5]:
import pandas as pd
from datetime import datetime
from textblob import TextBlob

# Create DataFrame from the provided table
data = {
    'Country': ['HK', 'SG', 'HK', 'SG', 'HK', 'SG'],
    'Channel': ['Internet', 'Internet', 'Internet', 'Mobile', 'Mobile', 'Mobile'],
    'Month_Year': ['Dec-23', 'Jan-24', 'Feb-24', 'Dec-23', 'Jan-24', 'Feb-24'],
    'Successful': [673480, 630152, 777223, 6751532, 6063108, 7002254],
    'UnSuccessful': [16516, 17966, 18763, 238105, 224842, 235435],
    'Total': [689996, 648118, 795986, 6989637, 6287950, 7237689],
    'Success Percentage': [99.9, 95.8, 99.87, 99.9, 99.87, 99.87]
}

df = pd.DataFrame(data)

# Convert Month_Year column to datetime format
df['Month_Year'] = pd.to_datetime(df['Month_Year'], format='%b-%y')

# Define NLG function to generate summary
def generate_summary(data):
    summary = []
    
    # Group data by country
    grouped_data = data.groupby('Country')

    # Iterate over each country
    for country, country_data in grouped_data:
        summary.append(f"Summary for {country}:")
        
        # Check if latest month success percentage is significantly above or below the threshold
        latest_month = country_data['Month_Year'].max()
        latest_data = country_data[country_data['Month_Year'] == latest_month]
        for _, row in latest_data.iterrows():
            if row['Success Percentage'] > 99.67:
                sentiment = 'achieved'
            else:
                sentiment = 'slightly missed'
            summary.append(f"Last month, {latest_month.strftime('%b-%y')}, target is {sentiment} for {row['Channel']} Channel.")

        # Analyze success percentage trends for each channel over the past three months
        for channel, channel_data in country_data.groupby('Channel'):
            trend_data = channel_data.tail(3)
            min_success_percentage = trend_data['Success Percentage'].min()
            for _, row in trend_data.iterrows():
                if row['Success Percentage'] == min_success_percentage:
                    summary.append(f"From the past three months for {channel} Channel, {row['Month_Year'].strftime('%b-%y')} shows the lowest Success Percentage.")

            # Analyze total volume trends for each channel over the past three months
            total_volumes = trend_data['Total'].tolist()
            max_volume_month = trend_data.loc[trend_data['Total'].idxmax(), 'Month_Year']
            min_volume_month = trend_data.loc[trend_data['Total'].idxmin(), 'Month_Year']
            if max_volume_month != min_volume_month:
                summary.append(f"From the past three months for {channel} Channel, {max_volume_month.strftime('%b-%y')} has the highest total volume, and {min_volume_month.strftime('%b-%y')} has the lowest total volume.")
    
    return summary

# Generate summary
summary = generate_summary(df)

# Print summary
for s in summary:
    print(s)


Summary for HK:
Last month, Feb-24, target is achieved for Internet Channel.
From the past three months for Internet Channel, Feb-24 shows the lowest Success Percentage.
From the past three months for Internet Channel, Feb-24 has the highest total volume, and Dec-23 has the lowest total volume.
From the past three months for Mobile Channel, Jan-24 shows the lowest Success Percentage.
Summary for SG:
Last month, Feb-24, target is achieved for Mobile Channel.
From the past three months for Internet Channel, Jan-24 shows the lowest Success Percentage.
From the past three months for Mobile Channel, Feb-24 shows the lowest Success Percentage.
From the past three months for Mobile Channel, Feb-24 has the highest total volume, and Dec-23 has the lowest total volume.
